# Setup and Installations

In [ ]:
!pip install --upgrade git+https://github.com/huggingface/transformers.git
!pip install -q datasets
!pip install matplotlib
!pip install -U transformers
!pip install scikit-learn pillow torchvision opencv-python
!pip install tensorboardX
!pip install torch
!pip install numpy
!pip install pandas
!pip install -U accelerate

# In Google Colab, use /content folder for uploading lump.zip

In [ ]:
!unzip lump.zip

# Data Loading and Preparation

In [ ]:

from datasets import load_dataset, DatasetDict, Dataset
from sklearn.model_selection import train_test_split
from PIL import Image
import torchvision.transforms as transforms
import pandas as pd
import numpy as np

# Load and prepare dataset
csv_file = 'lump/merged.csv'  # Modify this path as needed
df = pd.read_csv(csv_file)
df.columns = df.columns.str.strip()
print("Columns in the DataFrame:", df.columns)

# Define classes for acne classification
class_labels = ['blackheads', 'dark_spot', 'nodules', 'papules', 'pustules', 'whiteheads']

# Convert DataFrame to Dataset
dataset = Dataset.from_pandas(df)

# Split the dataset into train, validation, and test sets
ds = dataset.train_test_split(test_size=0.3)  # 70% train, 30% test
ds_test = ds['test'].train_test_split(test_size=0.5)  # 30% test --> 15% valid, 15% test

prepared_ds = DatasetDict({
    'train': ds['train'],
    'test': ds_test['test'],
    'valid': ds_test['train']
})

del ds_test

print(prepared_ds)


# Data Transformation

In [ ]:
from PIL import Image
import torchvision.transforms as transforms
import torch
from transformers import AutoImageProcessor, ViTForImageClassification
import transformers
class_labels = ['blackheads', 'dark spot', 'nodules', 'papules', 'pustules', 'whiteheads']

def transform(example_batch):
    desired_size = (224, 224)
    images = []
    root_dir = 'lump/images'
    # Load and resize images
    for img_path in example_batch['filename']:
        img = Image.open(f"{root_dir}/{img_path}").convert("RGB")
        img_resized = transforms.Resize(desired_size)(img)
        images.append(img_resized)

    # Process images with the processor
    inputs = processor(images, return_tensors='pt')

    # One-hot encode the labels
    labels_batch = {k.strip(): example_batch[k.strip()] for k in example_batch.keys() if k.strip() in class_labels}
    labels_matrix = torch.zeros((len(images), len(class_labels)))

    for idx, label in enumerate(class_labels):
        labels_matrix[:, idx] = torch.tensor(labels_batch[label])

    # Add labels to the inputs
    inputs['labels'] = labels_matrix
    # print(inputs)
    return inputs

# Apply the transform function to the dataset
processor = AutoImageProcessor.from_pretrained("facebook/convnextv2-tiny-1k-224")
prepared_ds = ds.with_transform(transform)
print(prepared_ds)

In [ ]:
def collate_fn(batch):
    pixel_values = torch.stack([x['pixel_values'] for x in batch])
    # print(pixel_values)
    # print([x['labels'] for x in batch])
    labels = torch.stack([x['labels'] for x in batch])
    return {
        'pixel_values': pixel_values,
        'labels': labels
    }

# Model Training and Saving
Adjust num_train_epochs in Training Arguments to higher value for more fit.
Note: save_model_to_bestmodels does not evaluate accuracy, it only saves the trained model.

In [ ]:

from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
import os

cl = ['blackheads', 'dark_spot', 'nodules', 'papules', 'pustules', 'whiteheads']

# Function to save models to 'bestmodels' directory
def save_model_to_bestmodels(model_id, model_name):
    print(f"Training and evaluating model: {model_id}")

    processor = AutoImageProcessor.from_pretrained(model_id)
    model = AutoModelForImageClassification.from_pretrained(
        model_id,
        num_labels=6,
        id2label={str(i): c for i, c in enumerate(cl)},
        label2id={c: str(i) for i, c in enumerate(cl)},
        ignore_mismatched_sizes=True,
        problem_type="multi_label_classification"
    )

    training_args = TrainingArguments(
        output_dir=f"bestmodels/{model_name}",
        per_device_train_batch_size=16,
        evaluation_strategy="no",
        save_strategy="epoch",
        fp16=True,
        num_train_epochs=1,
        logging_steps=500,
        learning_rate=2e-4,
        save_total_limit=1,
        remove_unused_columns=False,
        push_to_hub=False,
        report_to='tensorboard',
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=collate_fn,
        train_dataset=prepared_ds["train"],
        tokenizer=processor,
    )

    train_results = trainer.train()
    trainer.save_model(f"bestmodels/{model_name}")
    print(f"Model saved to 'bestmodels/{model_name}'")

# Create 'bestmodels' directory if it doesn't exist
os.makedirs("bestmodels", exist_ok=True)

# Models to train and save
model_list = [
    ("google/vit-base-patch16-224", "vit"),
    ("openai/clip-vit-base-patch32", "clip"),
    ("google/siglip-base-patch16-224", "siglip"),
    ("facebook/convnextv2-tiny-1k-224", "convnext"),
    ("apple/mobilevitv2-1.0-imagenet1k-256", "mobilevit"),
    ("google/mobilenet_v1_1.0_224", "mobilenet")
]

# Train and save each model
for model_id, model_name in model_list:
    save_model_to_bestmodels(model_id, model_name)

